[![Abrir en Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pugapatricia/gestion-documentaria-para-pymes/blob/main/etiquetado/Etiquetado_openai.ipynb)

[![Ver en GitHub](https://img.shields.io/badge/GitHub-Repo-black?logo=github)](https://github.com/pugapatricia/gestion-documentaria-para-pymes/tree/main/etiquetado)

#Importaciones

In [1]:
!pip install -q PyPDF2 python-docx openpyxl python-pptx xlrd transformers office365-rest-python-client msal requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 5.8 MB/s eta 0:00:00


In [2]:
import os
import io
import re
import json
from pathlib import Path
from PyPDF2 import PdfReader
import docx
import openpyxl
from pptx import Presentation
import xlrd
from transformers import pipeline
from office365.sharepoint.client_context import ClientContext
from office365.runtime.auth.user_credential import UserCredential
import os
import requests
import msal
import csv
import getpass
from openai import OpenAI

# Configuración

In [31]:
api_key = getpass.getpass("Introduce tu OpenAI API Key: ")
client = OpenAI(api_key=api_key)

Introduce tu OpenAI API Key: ··········


In [4]:
CLIENT_ID = "e3f2393e-7348-47d1-9c64-8d8efe6a5e95"  # tu nuevo Client ID
AUTHORITY = "https://login.microsoftonline.com/consumers"
SCOPE = ["User.Read", "Files.ReadWrite"]
ext_permitidas = {"pdf", "docx", "xlsx", "xls", "pptx", "txt", "csv"}
url = "https://graph.microsoft.com/v1.0/me/drive/root:/Etiquetados:/children"

# Conección con OneDrive


In [5]:
app = msal.PublicClientApplication(CLIENT_ID, authority=AUTHORITY)

flow = app.initiate_device_flow(scopes=SCOPE)
if "user_code" not in flow:
    raise Exception("No se pudo iniciar el device flow. Revisa tu configuración en Azure.")

print(flow["message"])  # 👉 Copia el código en https://microsoft.com/devicelogin
result = app.acquire_token_by_device_flow(flow)

if "access_token" not in result:
    raise Exception(f"Error autenticación: {result.get('error_description')}")

access_token = result["access_token"]
headers = {"Authorization": f"Bearer {access_token}"}

# Llamada a la API con tu token de acceso
resp = requests.get(url, headers=headers)
if resp.status_code != 200:
    raise Exception(f"Error al obtener archivos: {resp.text}")
data = resp.json()

To sign in, use a web browser to open the page https://www.microsoft.com/link and enter the code UVJ6L68F to authenticate.


#Funciones

Clasificador OpenAI


In [47]:
etiquetas = [
    "contrato", "arrendamiento", "vivienda", "modelo", "legal", "inmobiliaria",
    "admitidos", "cursos", "monográficos", "educación", "registro",
    "reserva", "inmueble", "alquiler", "NDA", "confidencialidad", "acuerdo", "plantilla",
    "unilateral", "UE", "OEPM", "presentación", "anexo", "personal", "informe",
    "análisis", "estados financieros", "contabilidad", "proyecto",
    "biblioteca", "datos", "csv", "inventario", "opción de compra", "freelancer",
    "formulario", "registro", "administrativo", "prueba", "test", "justificante",
    "bachillerato", "HH-CCSS", "temporada", "finca rústica", "rescisión",
    "empresa", "académico", "prórroga", "proyecto",
    "2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019",
    "2020", "2021", "2022", "2023", "2024", "2025", "2026", "2027", "2028", "2029", "2030"
]

def limpiar_texto(texto: str) -> str:
    if not texto:
        return ""
    texto = re.sub(r'\d+\n', '', texto)       # eliminar numeración de página
    texto = re.sub(r'\s+', ' ', texto)        # unir saltos de línea
    # conservar comas, puntos, guiones y paréntesis
    texto = re.sub(r'[^a-zA-Záéíóúüñ0-9\s,.\-()/:]', '', texto)
    return texto.strip()


def etiquetar_texto(texto, max_etiquetas=5, modelo="gpt-5-mini"):

    prompt = f"""
    Analiza el siguiente texto y selecciona hasta {max_etiquetas} etiquetas de la lista dada
    que mejor representen los temas principales. No inventes etiquetas fuera de la lista.

    Lista de etiquetas disponibles: {', '.join(etiquetas)}

    Texto: \"\"\"{texto}\"\"\"

    Devuelve solo las etiquetas seleccionadas, separadas por comas.
    """

    respuesta = client.chat.completions.create(
        model=modelo,
        messages=[{"role": "user", "content": prompt}],
        max_completion_tokens=150,
        temperature=1
    )

    etiquetas = respuesta.choices[0].message.content.strip()
    # Filtrar solo etiquetas válidas
    etiquetas = [e.strip() for e in etiquetas.split(",") if e.strip() in etiquetas]
    return etiquetas

Lector de documentos

In [48]:
def leer_pdf(contenido, limite_palabras=None):
    texto = ""
    reader = PdfReader(io.BytesIO(contenido))
    for page in reader.pages:
        page_text = page.extract_text()
        if page_text:
            texto += page_text + "\n"
    # Aplicar límite de palabras al final
    if limite_palabras:
        texto = " ".join(texto.split()[:limite_palabras])
    return texto

def leer_docx(contenido, limite_palabras=None):
    texto = ""
    doc = docx.Document(io.BytesIO(contenido))
    # Párrafos
    for p in doc.paragraphs:
        if p.text.strip():
            texto += p.text + "\n"
    # Tablas
    for table in doc.tables:
        for row in table.rows:
            row_text = " ".join([cell.text for cell in row.cells if cell.text.strip()])
            if row_text:
                texto += row_text + "\n"
    # Aplicar límite al final
    if limite_palabras:
        texto = " ".join(texto.split()[:limite_palabras])
    return texto

def leer_excel(contenido, limite_palabras=None):
    texto = ""
    wb = openpyxl.load_workbook(io.BytesIO(contenido), data_only=True, read_only=True)
    for sheet in wb.worksheets:
        for row in sheet.iter_rows(values_only=True):
            row_text = " ".join([str(cell) for cell in row if cell])
            if row_text:
                texto += row_text + "\n"
    if limite_palabras:
        texto = " ".join(texto.split()[:limite_palabras])
    return texto

def leer_xls(contenido, limite_palabras=None):
    texto = ""
    temp_file = "temp.xls"
    with open(temp_file, "wb") as f:
        f.write(contenido)
    wb = xlrd.open_workbook(temp_file)
    for sheet in wb.sheets():
        for row_idx in range(sheet.nrows):
            row = sheet.row_values(row_idx)
            row_text = " ".join([str(cell) for cell in row if cell])
            if row_text:
                texto += row_text + "\n"
    os.remove(temp_file)
    if limite_palabras:
        texto = " ".join(texto.split()[:limite_palabras])
    return texto

def leer_pptx(contenido, limite_palabras=None):
    texto = ""
    temp_file = "temp.pptx"
    with open(temp_file, "wb") as f:
        f.write(contenido)
    prs = Presentation(temp_file)
    for slide in prs.slides:
        for shape in slide.shapes:
            if hasattr(shape, "text") and shape.text.strip():
                texto += shape.text + "\n"
    os.remove(temp_file)
    if limite_palabras:
        texto = " ".join(texto.split()[:limite_palabras])
    return texto

def leer_txt_csv(contenido, limite_palabras=None):
    texto = contenido.decode("utf-8", errors="ignore")
    if limite_palabras:
        texto = " ".join(texto.split()[:limite_palabras])
    return texto

def leer_archivo(nombre, contenido, limite_palabras=None):
    ext = nombre.split(".")[-1].lower()
    if ext == "pdf":
        return leer_pdf(contenido, limite_palabras)
    elif ext == "docx":
        return leer_docx(contenido, limite_palabras)
    elif ext == "xlsx":
        return leer_excel(contenido, limite_palabras)
    elif ext == "xls":
        return leer_xls(contenido, limite_palabras)
    elif ext == "pptx":
        return leer_pptx(contenido, limite_palabras)
    elif ext in {"txt", "csv"}:
        return leer_txt_csv(contenido, limite_palabras)
    else:
        return ""

In [49]:
texto = leer_archivo(nombre, file_bytes, limite_palabras)

# Mostrar las primeras 500-1000 palabras para revisar
print(f"\n--- Texto extraído ({nombre}) ---")
print(texto[:1000])  # Ajusta la cantidad según lo que quieras ver
print("--- Fin del texto extraído ---\n")



--- Texto extraído (2023_05-Modelo_Documento_reserva_inmueble_en_alquiler_v.reducida.docx) ---
DOCUMENTO DE RESERVA DE INMUEBLE EN ARRENDAMIENTO En [lugar], [fecha] REUNIDOS De una parte, D./Dña. [nombre y apellidos del propietario], mayor de edad, de nacionalidad [nacionalidad], con domicilio en [domicilio] y DNI/NIE/pasaporte de su nacionalidad número [número identificación]. Actúa en su propio nombre y representación. En adelante, la “Parte Propietaria”. Y de otra, D./Dña. [nombre y apellidos inquilino 1], mayor de edad, de nacionalidad [nacionalidad], con domicilio en [domicilio] y DNI/NIE/pasaporte de su nacionalidad número [número identificación]. Actúa en su propio nombre y representación. D./Dña. [nombre y apellidos inquilino 2], mayor de edad, de nacionalidad [nacionalidad], con domicilio en [domicilio] y DNI/NIE/pasaporte de su nacionalidad número [número identificación]. Actúa en su propio nombre y representación. En adelante, la “Parte Reservante”. La Parte Propietaria y l

# Leer archivos de OneDrive

In [52]:
# Limitar a los primeros 3 archivos permitidos
archivos_a_procesar = archivos_permitidos[:3]

if archivos_a_procesar:
    resultados = {}

    for item in archivos_a_procesar:
        nombre = item["name"]
        download_url = item["@microsoft.graph.downloadUrl"]

        print(f"Procesando archivo: {nombre}")

        # Descargar contenido del archivo
        file_bytes = requests.get(download_url).content

        # Definir límite de palabras
        limite_palabras = 500  # Ajustable

        # Leer el archivo según su extensión y límite de palabras
        texto = leer_archivo(nombre, file_bytes, limite_palabras)

        # --- LIMPIEZA DEL TEXTO ---
        texto = limpiar_texto(texto)

        # Etiquetar texto si no está vacío
        if texto:
            etiquetas_detectadas = etiquetar_texto(texto)
        else:
            etiquetas_detectadas = []

        resultados[nombre] = etiquetas_detectadas

        # Mostrar resultado
        print(f"{nombre} → {etiquetas_detectadas}")

Procesando archivo: 11_07_2019_modelo_orientativo_de_contrato_de_arrendamiento_de_vivienda.pdf
11_07_2019_modelo_orientativo_de_contrato_de_arrendamiento_de_vivienda.pdf → []
Procesando archivo: 2016-admitidos_Segundo ciclo- Cursos monográficos.xls
2016-admitidos_Segundo ciclo- Cursos monográficos.xls → []
Procesando archivo: 2023_05-Modelo_Documento_reserva_inmueble_en_alquiler_v.reducida.docx
2023_05-Modelo_Documento_reserva_inmueble_en_alquiler_v.reducida.docx → []
Resultados guardados en etiquetas_onedrive.csv


#Guardar resultados

In [ ]:
    # Guardar todos los resultados en CSV
    with open("etiquetas_onedrive.csv", "w", newline="", encoding="utf-8-sig") as f:
        writer = csv.writer(f)
        writer.writerow(["Archivo", "Etiquetas"])
        for nombre, etiquetas_detectadas in resultados.items():
            etiquetas_str = ", ".join(etiquetas_detectadas) if etiquetas_detectadas else "Sin etiquetas"
            writer.writerow([nombre, etiquetas_str])

    print("Resultados guardados en etiquetas_onedrive.csv")

else:
    print("No se encontraron archivos permitidos para procesar.")

## Codigo para cargar en la etiqueta de descripcion en onedrive

In [ ]:
"""
# ============================
# Configuración
# ============================
json_path = "etiquetas_onedrive.json"
headers = {"Authorization": f"Bearer {access_token}"}

# Carpeta objetivo en OneDrive
carpeta_objetivo = "Etiquetados"

# ============================
# Cargar JSON de etiquetas
# ============================
with open(json_path, "r", encoding="utf-8") as f:
    etiquetas_data = json.load(f)

# ============================
# Aplicar etiquetas en la descripción
# ============================
for archivo, etiquetas in etiquetas_data.items():
    if not etiquetas:
        continue

    # Buscar archivo en OneDrive
    url_file = f"https://graph.microsoft.com/v1.0/me/drive/root:/{carpeta_objetivo}/{archivo}"
    resp_file = requests.get(url_file, headers=headers)

    if resp_file.status_code != 200:
        print(f"⚠️ No se encontró {archivo} en OneDrive")
        continue

    file_id = resp_file.json()["id"]

    # Guardamos etiquetas en el campo "description"
    url_update = f"https://graph.microsoft.com/v1.0/me/drive/items/{file_id}"
    payload = {"description": ", ".join(etiquetas)}

    response_update = requests.patch(
        url_update,
        headers={**headers, "Content-Type": "application/json"},
        json=payload
    )

    if response_update.status_code in [200, 204]:
        print(f"✅ Etiquetas {etiquetas} aplicadas a {archivo} en la descripción")
    else:
        print(f"⚠️ Error al actualizar {archivo}: {response_update.text}")
"""